In [3]:
class SVRM_ls:
    
    def __init__(self, C = 0.1, kernel = "linear", **kernel_param):
        import numpy as np
        from numpy.linalg import inv
        from sklearn.metrics.pairwise import pairwise_kernels
        from sklearn.utils import check_X_y, check_array 
        self.C = C
        self.kernel = kernel
        self.pairwise_kernels = pairwise_kernels
        self.kernel_param = kernel_param
        self.check_X_y = check_X_y
        self.check_array = check_array
        self.inv = inv
        
    def fit(self, X, y):
        X, y = self.check_X_y(X, y)
        a = np.unique(y); c = np.array([-1, 1])
        y = np.where(y == a[0], c[0], c[1])
        self.c = c; self.a = a
        # hyperparameters
        C = self.C 
        
        kernel = self.kernel
        pairwise_kernels = self.pairwise_kernels
        inv = self.inv
        
        # omega + upsilon
        omega_ = pairwise_kernels(X, X, kernel, **self.kernel_param) 
        + np.identity(y.size)*((y**2)/C)
        # ones vector
        onev = np.ones(y.shape).reshape(-1, 1)
        y = y.reshape(-1, 1)
        
        # solve for parameters
        A = np.linalg.pinv(np.block([[0, onev.T],[onev, omega_]]))
        B = np.concatenate((np.array([0]),y.reshape(-1)))
        sol =  A @ B
        
        b = sol[0]
        alpha = sol[1:]
        
        self.X = X
        self.alpha = alpha; self.b = b
        return self
        
    def predict(self, X_):
        pairwise_kernels = self.pairwise_kernels
        kernel_param = self.kernel_param
        kernel = self.kernel
        alpha = self.alpha
        b = self.b
        X = self.X
        # rename label variables
        c = self.c; a = self.a
        
        X_ = self.check_array(X_)
        predict = alpha @ pairwise_kernels(X, X_, metric = kernel, **kernel_param) + b
        predict1 = np.sign(predict)
        predict2 = np.where(predict1 == c[0], a[0], a[1])
        return predict2
    
    
        # coefficient
    def coef_(self):
        if self.kernel == "linear":
            alpha = self.alpha; X = self.X
            w = alpha @ X
            return w, self.b
        else: 
            return self.alpha,  self.b, self.X

In [5]:
class plott:
    @staticmethod
    def line(X, w, b):
        xmin, xmax = min(X[:,0])-1, max(X[:,0]+1)
        X_ = np.arange(xmin, xmax, 0.1)
    
        plt.plot(X_,(-w[0]*X_-b)/w[1])
    
    @staticmethod
    def scatter(X, y, m = "o"):
        plt.scatter(X[:,0], X[:,1], c = y, marker = m)

In [20]:
class utilss:
    @staticmethod
    def contour_plot(X, y, kernel="linear", C = 0.1, **kernel_param):
        h = .02
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),np.arange(y_min, y_max, h))
        
        Z = SVM_ls(C = C, kernel = kernel, **kernel_param).fit(X, y).predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
        
    @staticmethod
    def X_random(X, n_predict):
        max_min = np.array([[X[:,k].min(), X[:,k].max()] for k in range(X.shape[1])])
        X_test0 = np.random.choice(np.arange(round(max_min[0][0]), round(max_min[0][1]), 0.1), n_predict)
        max_min = np.delete(np.array(max_min),0,  axis = 0)
        for i in max_min:
            X_test1 = np.random.choice(np.arange(round(i[0]), round(i[1]), 0.1), n_predict)
            X_test0 = np.c_[X_test0, X_test1]
            X_test0.astype("float64")
        return X_test0